#### création des classes

In [1]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship


Base = declarative_base()


class Client(Base):
    __tablename__ = 'clients'
    # clé primaire
    id = Column(Integer, primary_key=True)  
    nom = Column(String)
    # relation 1-N
    commandes = relationship("Commande", back_populates="client")  


class Commande(Base):
    __tablename__ = 'commandes'
    id = Column(Integer, primary_key=True)
    montant = Column(Integer)
    # clé étrangère
    client_id = Column(Integer, ForeignKey('clients.id'))  
    client = relationship("Client", back_populates="commandes")

#### Initialisation de la base de données

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# 1. Connexion à une base de données SQLite :
# L'argument 'sqlite:///ma_base_clients.db' crée un fichier 'ma_base_clients.db' 
# dans le répertoire courant s'il n'existe pas.
engine = create_engine("sqlite:///ma_base_clients.db", echo=True)

# 2. Création des tables dans la base de données
# Cette ligne regarde toutes les classes qui héritent de Base (Client, Commande) 
# et crée les tables SQL correspondantes (si elles n'existent pas déjà).
Base.metadata.create_all(engine)

# 3. Configuration de la session :
# Lie la session à notre moteur (engine) pour pouvoir interagir avec la BDD.
Session = sessionmaker(bind=engine)
session = Session()

2025-11-27 14:58:48,204 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,204 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("clients")
2025-11-27 14:58:48,205 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:58:48,206 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("clients")
2025-11-27 14:58:48,206 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:58:48,207 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("commandes")
2025-11-27 14:58:48,207 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:58:48,208 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("commandes")
2025-11-27 14:58:48,209 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-27 14:58:48,209 INFO sqlalchemy.engine.Engine 
CREATE TABLE clients (
	id INTEGER NOT NULL, 
	nom VARCHAR, 
	PRIMARY KEY (id)
)


2025-11-27 14:58:48,210 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2025-11-27 14:58:48,219 INFO sqlalchemy.engine.Engine 
CREATE TABLE commandes (
	id INTEGER 

##### Ajouter des données


In [3]:
nouveau_client = Client(nom="Alice")
session.add(nouveau_client)
session.commit()


commande = Commande(montant=200, client=nouveau_client)
session.add(commande)
session.commit()

2025-11-27 14:58:48,238 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,239 INFO sqlalchemy.engine.Engine INSERT INTO clients (nom) VALUES (?)
2025-11-27 14:58:48,240 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('Alice',)
2025-11-27 14:58:48,243 INFO sqlalchemy.engine.Engine COMMIT
2025-11-27 14:58:48,255 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,257 INFO sqlalchemy.engine.Engine SELECT clients.id AS clients_id, clients.nom AS clients_nom 
FROM clients 
WHERE clients.id = ?
2025-11-27 14:58:48,258 INFO sqlalchemy.engine.Engine [generated in 0.00062s] (1,)
2025-11-27 14:58:48,259 INFO sqlalchemy.engine.Engine INSERT INTO commandes (montant, client_id) VALUES (?, ?)
2025-11-27 14:58:48,260 INFO sqlalchemy.engine.Engine [generated in 0.00057s] (200, 1)
2025-11-27 14:58:48,262 INFO sqlalchemy.engine.Engine COMMIT


#### Lire des données

In [4]:
# Lire toutes les commandes d'un client
client = session.query(Client).filter_by(nom="Alice").first()
print("-"*30)
print("--- Résultat de la requête ---")
for cmd in client.commandes:
    print("-"*30)
    print(cmd.montant)
    print("-"*30)

2025-11-27 14:58:48,278 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,280 INFO sqlalchemy.engine.Engine SELECT clients.id AS clients_id, clients.nom AS clients_nom 
FROM clients 
WHERE clients.nom = ?
 LIMIT ? OFFSET ?
2025-11-27 14:58:48,281 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ('Alice', 1, 0)
------------------------------
--- Résultat de la requête ---
2025-11-27 14:58:48,283 INFO sqlalchemy.engine.Engine SELECT commandes.id AS commandes_id, commandes.montant AS commandes_montant, commandes.client_id AS commandes_client_id 
FROM commandes 
WHERE ? = commandes.client_id
2025-11-27 14:58:48,284 INFO sqlalchemy.engine.Engine [generated in 0.00069s] (1,)
------------------------------
200
------------------------------


### Modifier / Supprimer
#### Modifier

In [5]:
commande_a_modifier = session.query(Commande).filter_by(id=1).first()

# Modifier un montant
commande_a_modifier.montant = 300
session.commit()

2025-11-27 14:58:48,293 INFO sqlalchemy.engine.Engine SELECT commandes.id AS commandes_id, commandes.montant AS commandes_montant, commandes.client_id AS commandes_client_id 
FROM commandes 
WHERE commandes.id = ?
 LIMIT ? OFFSET ?
2025-11-27 14:58:48,294 INFO sqlalchemy.engine.Engine [generated in 0.00070s] (1, 1, 0)
2025-11-27 14:58:48,295 INFO sqlalchemy.engine.Engine UPDATE commandes SET montant=? WHERE commandes.id = ?
2025-11-27 14:58:48,297 INFO sqlalchemy.engine.Engine [generated in 0.00117s] (300, 1)
2025-11-27 14:58:48,300 INFO sqlalchemy.engine.Engine COMMIT


##### Vérification

In [6]:
# Lire toutes les commandes d'un client
client = session.query(Client).filter_by(nom="Alice").first()
print("-"*30)
print("--- Résultat de la requête ---")
for cmd in client.commandes:
    print("-"*30)
    print(cmd.montant)
    print("-"*30)

2025-11-27 14:58:48,317 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,318 INFO sqlalchemy.engine.Engine SELECT clients.id AS clients_id, clients.nom AS clients_nom 
FROM clients 
WHERE clients.nom = ?
 LIMIT ? OFFSET ?
2025-11-27 14:58:48,318 INFO sqlalchemy.engine.Engine [cached since 0.03837s ago] ('Alice', 1, 0)
------------------------------
--- Résultat de la requête ---
2025-11-27 14:58:48,320 INFO sqlalchemy.engine.Engine SELECT commandes.id AS commandes_id, commandes.montant AS commandes_montant, commandes.client_id AS commandes_client_id 
FROM commandes 
WHERE ? = commandes.client_id
2025-11-27 14:58:48,320 INFO sqlalchemy.engine.Engine [cached since 0.03697s ago] (1,)
------------------------------
300
------------------------------


#### Supprimer

In [7]:
# Supprimer une commande
session.delete(commande_a_modifier)
session.commit()

2025-11-27 14:58:48,327 INFO sqlalchemy.engine.Engine DELETE FROM commandes WHERE commandes.id = ?
2025-11-27 14:58:48,328 INFO sqlalchemy.engine.Engine [generated in 0.00065s] (1,)
2025-11-27 14:58:48,331 INFO sqlalchemy.engine.Engine COMMIT


##### Vérification

In [8]:
# Lire toutes les commandes d'un client
client = session.query(Client).filter_by(nom="Alice").first()
print("-"*30)
print("--- Résultat de la requête ---")
for cmd in client.commandes:
    print("-"*30)
    print(cmd.montant)
    print("-"*30)

2025-11-27 14:58:48,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,351 INFO sqlalchemy.engine.Engine SELECT clients.id AS clients_id, clients.nom AS clients_nom 
FROM clients 
WHERE clients.nom = ?
 LIMIT ? OFFSET ?
2025-11-27 14:58:48,351 INFO sqlalchemy.engine.Engine [cached since 0.07152s ago] ('Alice', 1, 0)
------------------------------
--- Résultat de la requête ---
2025-11-27 14:58:48,353 INFO sqlalchemy.engine.Engine SELECT commandes.id AS commandes_id, commandes.montant AS commandes_montant, commandes.client_id AS commandes_client_id 
FROM commandes 
WHERE ? = commandes.client_id
2025-11-27 14:58:48,353 INFO sqlalchemy.engine.Engine [cached since 0.06988s ago] (1,)


#### Insérer plusieurs commandes

In [9]:
# On suppose que nous avons déjà un Client existant avec id=1
nouveau_client = Client(nom="Alice")
session.add(nouveau_client)
session.commit()
# Il est préférable d'insérer le client en amont si ce n'est pas déjà fait.
client_existant = session.query(Client).filter_by(id=1).first()

if client_existant:
    # Création des nouveaux objets Commande en Python
    nouvelle_commande_1 = Commande(montant=50, client=client_existant)
    nouvelle_commande_2 = Commande(montant=150, client=client_existant)
    nouvelle_commande_3 = Commande(montant=25, client_id=client_existant.id) # On peut aussi utiliser directement le client_id

    # 1. Ajout des trois objets à la session en une seule fois (add_all)
    session.add_all([
        nouvelle_commande_1, 
        nouvelle_commande_2, 
        nouvelle_commande_3
    ])

    # 2. Le commit unique exécute toutes les commandes INSERT en base de données.
    session.commit()
    print("Trois nouvelles commandes ont été insérées en masse.")
else:
    print("Le client avec l'ID 1 n'a pas été trouvé. Veuillez d'abord l'insérer.")

2025-11-27 14:58:48,361 INFO sqlalchemy.engine.Engine INSERT INTO clients (nom) VALUES (?)
2025-11-27 14:58:48,362 INFO sqlalchemy.engine.Engine [cached since 0.123s ago] ('Alice',)
2025-11-27 14:58:48,365 INFO sqlalchemy.engine.Engine COMMIT
2025-11-27 14:58:48,375 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,376 INFO sqlalchemy.engine.Engine SELECT clients.id AS clients_id, clients.nom AS clients_nom 
FROM clients 
WHERE clients.id = ?
 LIMIT ? OFFSET ?
2025-11-27 14:58:48,377 INFO sqlalchemy.engine.Engine [generated in 0.00047s] (1, 1, 0)
2025-11-27 14:58:48,379 INFO sqlalchemy.engine.Engine INSERT INTO commandes (montant, client_id) VALUES (?, ?) RETURNING id
2025-11-27 14:58:48,380 INFO sqlalchemy.engine.Engine [generated in 0.00008s (insertmanyvalues) 1/3 (ordered; batch not supported)] (50, 1)
2025-11-27 14:58:48,381 INFO sqlalchemy.engine.Engine INSERT INTO commandes (montant, client_id) VALUES (?, ?) RETURNING id
2025-11-27 14:58:48,381 INFO sqlalchemy.en

#### lire toutes les commandes

In [10]:
# Récupération de tous les objets Commande de la table
toutes_les_commandes = session.query(Commande).all()

print(f"Nombre total de commandes trouvées : {len(toutes_les_commandes)}")

# Itérer sur la liste des objets récupérés
for commande in toutes_les_commandes:
    print(f"- Commande ID: {commande.id}, Montant: {commande.montant}, Client ID: {commande.client_id}, Client nom: {commande.client.nom}")

2025-11-27 14:58:48,399 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-27 14:58:48,401 INFO sqlalchemy.engine.Engine SELECT commandes.id AS commandes_id, commandes.montant AS commandes_montant, commandes.client_id AS commandes_client_id 
FROM commandes
2025-11-27 14:58:48,401 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ()
Nombre total de commandes trouvées : 3
2025-11-27 14:58:48,403 INFO sqlalchemy.engine.Engine SELECT clients.id AS clients_id, clients.nom AS clients_nom 
FROM clients 
WHERE clients.id = ?
2025-11-27 14:58:48,404 INFO sqlalchemy.engine.Engine [generated in 0.00056s] (1,)
- Commande ID: 1, Montant: 50, Client ID: 1, Client nom: Alice
- Commande ID: 2, Montant: 150, Client ID: 1, Client nom: Alice
- Commande ID: 3, Montant: 25, Client ID: 1, Client nom: Alice


### Fermer la connexion

In [12]:
engine.dispose()